# Generating BTS roads network in the USA

In [1]:
import geopandas as gpd
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd
import collections
import re
import numpy as np
import time

from shapely.geometry import Polygon, LineString, Point

In [2]:
pd.set_option('display.max_columns', None)
usa_roads = gpd.read_file("data/National_Highway_Network-shp/National_Highway_Planning_Network.shp")

In [3]:
usa_roads.head(2)

,OBJECTID,YEAR_RECOR,ROUTE_ID,BEGIN_POIN,END_POINT,F_SYSTEM,FACILITY_T,URBAN_CODE,RECTYPE,VERSION,RECID,ORIGID,CTFIPS,SOURCE,LGURB,SMURB,SIGN1,SIGNT1,SIGNN1,SIGNQ1,SIGN2,SIGNT2,SIGNN2,SIGNQ2,SIGN3,SIGNT3,SIGNN3,SIGNQ3,LNAME,MILES,KM,FCLASS,RUCODE,STATUS,NHS,STRAHNET,FAC_ID,CONN_ID,CONN_DES,CONN_MILES,LRSKEY,LRSSEQ,BEGMP,ENDMP,STFIPS,ShapeSTLen,geometry
0,1,2011,150000,157.020,176.061,3,2,99999,L,2014.05,2002148,2002148,290,H,0,0,S65,S,65,None,None,None,None,None,None,None,None,None,JAMES DALTON HWY,18.070,29.081,2,1,1,7,0,None,None,None,0.0,001500000000002,2,75.236,193.382,02,78290.607852,"LINESTRING (-150.28945 67.01801, -150.28940 67..."
1,2,2010,50,0.493,0.931,3,2,99998,L,2014.05,15001104,15001104,7,H,0,0,S50,S,50,None,None,None,None,None,None,None,None,None,KAUMUALII HWY,0.428,0.689,6,2,1,7,0,None,None,None,0.0,000000005000007,10,30.895,31.953,15,748.807358,"LINESTRING (-159.38840 21.96962, -159.38724 21..."


In [4]:
usa_roads.STATUS.value_counts()

1    625117
0      1200
2        49
Name: STATUS, dtype: int64

Note: I may have to remove the roads with status 0 (maybe abandonned) or 2 (maybe in construction)

## Set Highway speed for each state

In [5]:
state_map_to_STFIPS_table = pd.read_html('https://www.careerinfonet.org/links_st.asp?soccode=&stfips=&id=&nodeid=111')[0].iloc[:-1, :]
state_map_to_STFIPS_map = pd.Series(index=state_map_to_STFIPS_table['State Name'],data=state_map_to_STFIPS_table['STFIPS code'].astype('int').values)

In [6]:
speed_table = pd.read_html('https://en.wikipedia.org/wiki/Speed_limits_in_the_United_States')[1]
speed_table['State or territory'] = speed_table['State or territory'].str.extract('([a-zA-Z\s]+)')
speed_table['STFIPS code'] = speed_table['State or territory'].replace(state_map_to_STFIPS_map).astype('str').str.extract('(\d+)')
speed_table['Freeway (trucks)'] = speed_table['Freeway (trucks)'].str.extract('(\d+)').fillna('55')
speed_map = speed_table.loc[:,['STFIPS code','Freeway (trucks)']].dropna(axis=0).astype('int').rename(columns={"Freeway (trucks)": "speed_mph"})
speed_map.set_index('STFIPS code',drop=True,inplace=True)

speed_map_kmh = round(speed_map* 1.609344, 0).squeeze()
speed_map_kmh.head(2)

STFIPS code
1    113.0
2    105.0
Name: speed_mph, dtype: float64

In [7]:
usa_roads = usa_roads.drop(usa_roads.columns.difference(['KM','STFIPS','geometry','id']), axis=1)
usa_roads['speed_kmh'] = usa_roads['STFIPS'].astype('int').replace(speed_map_kmh)
usa_roads['duration'] = pd.eval('usa_roads.KM / usa_roads.speed_kmh')
usa_roads['key'] = 0
usa_roads = usa_roads.replace(to_replace='None', value=np.nan).dropna(subset=['geometry'])
usa_roads.head(2)

,KM,STFIPS,geometry,speed_kmh,duration,key
0,29.081,02,"LINESTRING (-150.28945 67.01801, -150.28940 67...",105.0,0.276962,0
1,0.689,15,"LINESTRING (-159.38840 21.96962, -159.38724 21...",89.0,0.007742,0


When taking the entire USA, the `STFIPS` can be used to map the max highway speed

In [ ]:
def add_incident_nodes(df_in): 
    df = df_in.copy()
    start = time.time()     
    u_values = [str((round(i.geometry.coords[:][0][0],4),round(i.geometry.coords[:][0][1],4))) for i in df.itertuples()]
    v_values = [str((round(i.geometry.coords[:][-1][0],4),round(i.geometry.coords[:][-1][1],4))) for i in df.itertuples()]
    
    df['u'] = u_values
    df['v'] = v_values
    
    df.drop('geometry',inplace = True, axis=1)
    print(f'time elapsed: {time.time()-start}')
    return df

def add_incident_nodes_fast(df_in): 
    df = df_in.copy()
    start = time.time()
    df['geometry'] = df.geometry.astype(str)
    df['u'] = df.geometry.str.extract('\((-?\d+\.\d+)')
    df['v'] = df.geometry.str.extract('(-?\d+\.\d+)\)$')
    
    df.drop('geometry',inplace = True, axis=1)
    print(f'time elapsed: {time.time()-start}')
    return df
       

usa_roads_slow = add_incident_nodes(usa_roads)


usa_roads_fast = add_incident_nodes_fast(usa_roads)


time elapsed: 174.76628518104553


In [ ]:
usa_roads_fast.tail()

In [ ]:
usa_roads = usa_roads_slow

In [ ]:
usa_roads['length'] = usa_roads['KM']*1000
usa_roads['CO2_eq_kg'] = pd.eval('usa_roads.length /1000 * 0.080513')

In [ ]:
def gen_nodes_gdfs(df):
    start = time.time()
    nodes_df = gpd.GeoDataFrame(columns=['nodes_pos','tag','x','y','osmid','geometry'],crs="EPSG:4326")
    
    nodes_df['nodes_pos'] = pd.unique(df[['u', 'v']].values.ravel('K'))
    
    pattern = re.compile(r'(-?\d+.\d+)')
    
    coords = nodes_df['nodes_pos'].str.extractall(pattern).unstack(level=-1)
    coords.columns = coords.columns.droplevel()
    coords.rename(columns = {0:'x',1:'y'},inplace=True)
    nodes_df['x'] = coords.x.astype(float)
    nodes_df['y'] = coords.y.astype(float)
    nodes_df['osmid'] = nodes_df.nodes_pos
    nodes_df['geometry'] = [Point(x,y) for x,y in zip(nodes_df.x,nodes_df.y)]
    
    nodes_df['tag'] = 'road'
    nodes_df['new_idx'] = range(1000000000,1000000000 + len(nodes_df))
    
    print(f'time elapsed: {time.time()-start}')
    nodes_df.set_index('nodes_pos', drop=True, inplace=True)
    return nodes_df

In [ ]:
nodes_gdfs = gen_nodes_gdfs(usa_roads)

In [ ]:
nodes_gdfs.head()

In [ ]:
nodes_gdfs.info()

In [ ]:
map_ids = nodes_gdfs.loc[:,'new_idx'].squeeze()

In [ ]:
G = ox.graph_from_gdfs(nodes_gdfs, usa_roads)
nx.relabel_nodes(G, dict(map_ids), copy=False)

In [ ]:
def get_edge_attribute(G, node=0):
    i,j = list(G.edges())[node]
    print(f'node_id is {i},{j}')
    return G.edges[i,j,0]
get_edge_attribute(G)

In [ ]:
G_s = ox.simplify_graph(G)

In [ ]:
fig, ax = ox.plot_graph(G,node_color='blue',bgcolor='white',node_alpha=0.5)

In [ ]:
G_undirected = G.to_undirected()
G_undirected_s = G_s.to_undirected()

component_size = sorted([len(component) for component in nx.connected_components(G_undirected)], reverse=True)
fig, ax = plt.subplots(figsize=(12,3))
plt.bar(range(len(component_size)),component_size)
plt.plot(range(len(component_size)),component_size,color='red',alpha=0.5)
plt.title('connected components size')

In [ ]:
largest_cc_nodes = max(nx.connected_components(G_undirected), key=len)
largest_cc = G_undirected.subgraph(largest_cc_nodes).copy()

largest_cc_nodes_s = max(nx.connected_components(G_undirected_s), key=len)
largest_cc_s = G_undirected_s.subgraph(largest_cc_nodes_s).copy()

It should be noted that by simplifying the graph, the edges attributes where added as list elements. 
For the duration attribute, theses element will have to be summed later on.

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)

In [ ]:
edges.head(5)